In [ ]:
import os
import time
import requests
import requests_cache

import numpy as np
import pandas as pd
from scipy.stats import linregress

from dotenv import load_dotenv
load_dotenv()

## GitHub User Data Scraping

### Testing GH Search API

In [ ]:
url = "https://api.github.com/search/users"

params = {
    'q': 'followers:>100 location:Toronto',  # Uses a query string (GraphQL).
    'per_page': 100,
    'page': 1
}
# Authenticated Users: 5,000 requests per hour.
access_token = os.getenv('GITHUB_TOKEN')
headers = {'Authorization': f'token {access_token}'}

response = requests.get(url, params=params, headers=headers)
print('URL:', response.url)

if response.status_code == 200:
    data = response.json()
    print('# users:', len(data['items']))
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

In [ ]:
response.links  # feature of requests library

In [ ]:
data.keys()

In [ ]:
data['items'][0]

### Search for Users

* Toronto users with more than 100 followers

In [ ]:
def dynamic_delay(response):
    if response.headers.get('X-RateLimit-Remaining') == '0':
        reset_time = int(response.headers.get('X-RateLimit-Reset'))
        sleep_time = reset_time - int(time.time()) + 5  # Add a buffer of 5s.

        print(f"Rate limit exceeded. Sleeping for {sleep_time} seconds!")
        time.sleep(sleep_time)

    time.sleep(1)  # Sleep for 1s regardless.

In [ ]:
printed_message_from_cached = False
requests_cache.install_cache('cache/search_users', expire_after=None)

users = []  # All users in Toronto with more than 100 followers.

url = "https://api.github.com/search/users"
params = {
    'q': 'followers:>100 location:Toronto',  # Uses a query string (GraphQL).
    'per_page': 100,
    'page': 1
}

access_token = os.getenv('GITHUB_TOKEN')
headers = {'Authorization': f'token {access_token}'}

while True:
    response = requests.get(url, params=params, headers=headers)

    if response.status_code != 200:
        print(f"Failed to fetch users. Status code: {response.status_code}")
        break

    data = response.json()

    for user in data.get('items', []):
        users.append({
            'login': user['login'],
            'id': user['id'],
            'url': user['url'],
            'repos_url': user['repos_url']
        })
    
    if response.from_cache:
        if not printed_message_from_cached:
            print('Fetched from cache.')
            printed_message_from_cached = True
    else:
        print("Fetched from API:", response.url)
        dynamic_delay(response)

    # Check if there are more pages.
    if 'next' not in response.links:
        break

    params['page'] += 1

print('# of users:', len(users))

In [ ]:
dict(response.headers)

In [ ]:
users[0]

### Fetch User Data

In [ ]:
def clean_company_name(company):
    company_name = None
    if company:
        company_name = company.strip()
        if company_name.startswith('@'):
            company_name = company_name[1:]
        company_name = company_name.upper()
    
    return company_name

In [ ]:
printed_message_from_cached = False
requests_cache.install_cache('cache/users', expire_after=None)

users_data = []

for user in users:
    response = requests.get(user['url'], headers=headers)

    if response.status_code != 200:
        print(f"Failed to fetch {user['login']}'s data. Status code: {response.status_code}")
        break
    
    data = response.json()
    
    users_data.append({
        'login': data['login'],
        'name': data['name'],
        'company': clean_company_name(data['company']),
        'location': data['location'],
        'email': data['email'],
        'hireable': data['hireable'],
        'bio': data['bio'],
        'public_repos': data['public_repos'],
        'followers': data['followers'],
        'following': data['following'],
        'created_at': data['created_at'],
    })

    if response.from_cache:
        if not printed_message_from_cached:
            print('Fetched from cache.')
            printed_message_from_cached = True
    else:
        print('Fetched from API:', response.url)
        dynamic_delay(response)

users_data[0]

In [ ]:
len(users_data)

In [ ]:
dict(response.headers)

#### Save user data to .csv

In [ ]:
users_df = pd.DataFrame(users_data)
users_df.head()

In [ ]:
total_number_of_repos_expected = users_df[users_df['public_repos'] <= 500]['public_repos'].sum() + (500 * len(users_df[users_df['public_repos'] > 500]))
print(f"Total Expected Repos: {total_number_of_repos_expected}")

In [ ]:
n_unique_users = len(users_df['login'].unique())  # Sanity check.
print(f"# of unique users: {n_unique_users}")

In [ ]:
users_df['hireable'].fillna(value=False, inplace=True)

In [ ]:
users_df.info()

In [ ]:
users_df.to_csv('users.csv', index=False)

### Fetch Users' Repo Data

In [ ]:
printed_message_from_cached = False
requests_cache.install_cache('cache/repos', expire_after=None)

repos_data = []

for user in users:
    repos = []
    params = {
        'sort': 'pushed',
        'direction': 'desc',
        'per_page': 100,
        'page': 1,
    }
    while True:
        response = requests.get(user['repos_url'], params=params, headers=headers)

        if response.status_code != 200:
            print(f"Failed to fetch { user['login'] }'s repo data. Status code: {response.status_code}")
            break
        
        data = response.json()
        for repo in data:
            repos.append({
                'login': user['login'],
                'full_name': repo['full_name'],
                'created_at': repo['created_at'],
                'stargazers_count': repo['stargazers_count'],
                'watchers_count': repo['watchers_count'],
                'language': repo['language'],
                'has_projects': repo['has_projects'],
                'has_wiki': repo['has_wiki'],
                'license_name': repo['license']['key'] if repo['license'] else None,
            })
        
        if response.from_cache:
            if not printed_message_from_cached:
                print('Fetched from cache.')
                printed_message_from_cached = True
        else:
            print('Fetched from API:', response.url)
            dynamic_delay(response)

        if ('next' not in response.links) or (params['page'] == 5):
            break

        params['page'] += 1
    
    repos_data.extend(repos)

print('# of Repos:', len(repos_data))

In [ ]:
print('Pass repos sanity check:', total_number_of_repos_expected == len(repos_data))

In [ ]:
repos_data[0]

#### Save repo data to .csv

In [ ]:
repos_df = pd.DataFrame(repos_data)
repos_df.head()

In [ ]:
repos_df.info()

In [ ]:
repos_df.to_csv('repositories.csv', index=False)

## Questions

In [ ]:
users_df = pd.read_csv('users.csv')
repos_df = pd.read_csv('repositories.csv')

Q1. Who are the top 5 users in Toronto with the highest number of followers? List their login in order, comma-separated.

In [ ]:
users_df.head()

In [ ]:
users_df.nlargest(n=5, columns='followers')[['login', 'followers']]

In [ ]:
','.join(users_df.nlargest(n=5, columns='followers')['login'].to_list())

Q2. Who are the 5 earliest registered GitHub users in Toronto? List their login in ascending order of created_at, comma-separated.

In [ ]:
users_df['created_at'] = pd.to_datetime(users_df['created_at'])
users_df.head()

In [ ]:
users_df.iloc[users_df['created_at'].sort_values(ascending=True).index[:5].to_list()][['login', 'created_at']]

In [ ]:
oldest_users = users_df.iloc[users_df['created_at'].sort_values(ascending=True).index[:5].to_list()]['login'].to_list()
','.join(sorted(oldest_users))

Q3. What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [ ]:
repos_df['created_at'] = pd.to_datetime(repos_df['created_at'])
repos_df.head()

In [ ]:
repos_df['license_name'].value_counts(dropna=True)[:3]

In [ ]:
','.join(repos_df['license_name'].value_counts(dropna=True).index[:3].to_list())

In [ ]:
repos_df['license_name'].isna().sum()

Q4. Which company do the majority of these developers work at?

In [ ]:
users_df['company'].value_counts()[:5]

In [ ]:
users_df['company'].value_counts().index[0]

Q5. Which programming language is most popular among these users?

In [ ]:
repos_df['language'].value_counts()[:5]

In [ ]:
repos_df['language'].value_counts().index[0]

Q6. Which programming language is the second most popular among users who joined after 2020?

In [ ]:
pd.merge?

In [ ]:
merged = repos_df.merge(users_df, how='inner', on='login', suffixes=('_repo', '_user'))
merged.head()

In [ ]:
users_df[users_df['created_at'].dt.year > 2020]['public_repos'].sum()

In [ ]:
merged[merged['created_at_user'].dt.year > 2020]['language'].value_counts()[:5]

In [ ]:
merged[merged['created_at_user'].dt.year > 2020]['language'].value_counts().index[1]

Q7. Which language has the highest average number of stars per repository?

In [ ]:
repos_df.groupby('language')['stargazers_count'].mean().sort_values(ascending=False)[:5]

In [ ]:
repos_df.groupby('language')['stargazers_count'].mean().sort_values(ascending=False).index[0]

Q8. Let's define `leader_strength` as `followers / (1 + following)`. Who are the top 5 in terms of `leader_strength`? List their login in order, comma-separated.

In [ ]:
users_df['leader_strength'] = users_df['followers'] / (1 + users_df['following'])
users_df.head()

In [ ]:
users_df.nlargest(5, 'leader_strength')[['login', 'leader_strength']]

In [ ]:
','.join(users_df.nlargest(5, 'leader_strength')['login'].to_list())

Q9. What is the correlation between the number of followers and the number of public repositories among users in Toronto?

In [ ]:
users_df['followers'].corr(users_df['public_repos']).round(3)

Q10. Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

In [ ]:
users_df[['public_repos', 'followers']].isna().sum()

In [ ]:
linregress(users_df['public_repos'], users_df['followers']).slope.round(3)

Q11. Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?

In [ ]:
has_projects = repos_df['has_projects'].astype(int)
has_wiki = repos_df['has_wiki'].astype(int)

In [ ]:
has_projects.corr(has_wiki).round(3)

In [ ]:
repos_df[['has_projects', 'has_wiki']].corr().round(3)

Q12. Do hireable users follow more people than those who are not hireable?

In [ ]:
users_df['hireable'].value_counts(dropna=False)

In [ ]:
users_df[users_df['hireable'].notnull()]['following'].mean()

In [ ]:
users_df[users_df['hireable'].isnull()]['following'].mean()

In [ ]:
(users_df[users_df['hireable'].notnull()]['following'].mean() - users_df[users_df['hireable'].isnull()]['following'].mean()).round(3)

In [ ]:
users_df['hireable'].value_counts(dropna=False)

In [ ]:
users_df.groupby('hireable')['following'].mean()

In [ ]:
q12 = users_df[['hireable', 'following']].copy()
q12.isna().sum()

In [ ]:
q12['hireable'].fillna(value=False, inplace=True)
q12.head()

In [ ]:
q12['hireable'].value_counts()

In [ ]:
(q12.groupby('hireable')['following'].mean()[True] - q12.groupby('hireable')['following'].mean()[False]).round(3)

Q13. Some developers write long bios. Does that help them get more followers? What's the correlation of the length of their bio (in Unicode characters) with followers? (Ignore people without bios)

In [ ]:
users_with_bio = users_df.loc[users_df['bio'].notnull(), :].copy().reset_index(drop=True, inplace=False)
users_with_bio['bio_length'] = users_with_bio['bio'].str.split().str.len()

In [ ]:
users_with_bio.tail()

In [ ]:
users_with_bio[['bio_length', 'followers']].head()

In [ ]:
linregress(users_with_bio['bio_length'], users_with_bio['followers']).slope.round(3)

Q14. Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

In [ ]:
repos_df['created_at'].dt.weekday.value_counts().sort_index()

In [ ]:
repos_df[repos_df['created_at'].dt.weekday >= 5].groupby('login').size().sort_values(ascending=False)[:5]

In [ ]:
','.join(repos_df[repos_df['created_at'].dt.weekday >= 5].groupby('login').size().sort_values(ascending=False)[:5].index.to_list())

Q15. Do people who are hireable share their email addresses more often?

[fraction of users with email when hireable=true] minus [fraction of users with email for the rest]

In [ ]:
users_df[(users_df['hireable'].notnull()) & (users_df['email'].notnull())].shape

In [ ]:
users_df[(users_df['email'].notnull())].shape

In [ ]:
round((users_df[(users_df['hireable'].notnull()) & (users_df['email'].notnull())].shape[0] / users_df.shape[0]) - (users_df[(users_df['email'].notnull())].shape[0] / users_df.shape[0]), 3)

In [ ]:
users_df['hireable']

Q16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)

In [ ]:
users_df['name'].str.strip().str.split().str[-1].value_counts()[:5]

In [ ]:
users_df['name'].str.strip().str.split().str[-1].value_counts().index[0]